<a href="https://colab.research.google.com/github/KusalaniR/Portfolio/blob/main/notebooks/chatbot_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
from google.api_core.exceptions import ResourceExhausted
import textwrap


In [2]:
# Supported languages
SUPPORTED_LANGUAGES = ["en", "si"]

In [3]:
#Language Detection Function
# ---------------------------------------------
# Detect user language (English / Sinhala)
# ---------------------------------------------

def detect_language(text):
    """
    Detects Sinhala vs English based on Unicode range.
    Sinhala characters range: \u0D80 - \u0DFF
    """
    for ch in text:
        if '\u0D80' <= ch <= '\u0DFF':
            return "si"
    return "en"


In [4]:
# ---------------------------------------------
# Sinhala medical question → English intent
# ---------------------------------------------

def sinhala_to_english_question(text):
    """
    Converts common Sinhala medical questions into English.
    Rule-based (safe & FYP-appropriate).
    """

    text = text.lower()

    if "mch" in text and "ඉහළ" in text:
        return "Why is my MCH high?"

    if "cholesterol" in text or "කොලෙස්ටරෝල්" in text:
        return "Is my cholesterol normal?"

    if "rdw" in text:
        return "What does RDW mean?"

    if "රතු රුධිර" in text:
        return "What do my red blood cell results mean?"

    return "Explain my blood report"


In [5]:
#Translation Function
# ---------------------------------------------
# Translate English → Sinhala (fallback version)
# ---------------------------------------------

def translate_to_sinhala(text):
    """
    Clean + medical-safe Sinhala translation
    """

    replacements = {
        "Based on the report provided,": "ඔබගේ රුධිර වාර්තාව අනුව",
        "there are two separate MCH test results listed,":
            "MCH පරීක්ෂණය සඳහා අගයන් දෙකක් සටහන් වී ඇත",
        "High": "ඉහළ",
        "Low": "අඩු",
        "Normal": "සාමාන්‍ය",
        "Mean Corpuscular Hemoglobin":
            "Mean Corpuscular Hemoglobin (MCH)",
        "red blood cells": "රතු රුධිර කණිකා",
        "This information is educational only.":
            "මෙය අධ්‍යාපනික විස්තරයක් පමණි.",
        "Please consult your healthcare provider.":
            "කරුණාකර වෛද්‍යවරයෙකු හමුවන්න.",
        "I cannot provide a medical diagnosis.":
            "මෙය වෛද්‍ය නිශ්චයයක් නොවේ."
    }

    for en, si in replacements.items():
        text = text.replace(en, si)

    return text




In [6]:


from google.colab import drive
drive.mount("/content/drive")

# Load extracted OCR + Gemini results (RAG source 1)
final_df = pd.read_csv(
    "/content/drive/MyDrive/MedGen.AI Datasets/FINALIZED DATASETS/OCR/extracted_report_results.csv"
)

final_df.head()


Mounted at /content/drive


,test_name,value,status,language,explanation
0,RBC,437.0,Unknown,si,\nTest: RBC\nValue: 437.0\nStatus: Unknown\n\n...
1,MCV,92.0,Normal,si,\nTest: MCV\nValue: 92.0\nStatus: Normal\n\nEx...
2,MCH,287.0,High,si,\nTest: MCH\nValue: 287.0\nStatus: High\n\nExp...
3,MCH,322.0,High,si,\nTest: MCH\nValue: 322.0\nStatus: High\n\nExp...
4,RDW,8.0,Low,si,\nTest: RDW\nValue: 8.0\nStatus: Low\n\nExplan...


In [7]:
#Load medical Knowledge (RAG source 2)
knowledge_df = pd.read_csv(
    "/content/drive/MyDrive/MedGen.AI Datasets/FINALIZED DATASETS/blood_test_knowledge.csv",
    encoding="latin1"
)

knowledge_df.head()


,test_name,normal_range,unit,low_meaning,high_meaning,simple_explanation_en
0,Hemoglobin,"1215.5 (Female), 13.517.5 (Male)",g/dL,May indicate low oxygen-carrying capacity,May indicate dehydration,Hemoglobin is a protein in red blood cells tha...
1,Hematocrit,"3646% (Female), 4153% (Male)",%,May indicate anemia,May indicate dehydration,Hematocrit shows the percentage of red blood c...
2,WBC Count,4.010.0,K/uL,May reduce ability to fight infections,May indicate infection or inflammation,White blood cells help your body fight infecti...
3,Red Blood Cells,"4.25.4 (Female), 4.76.1 (Male)",m/uL,May reduce oxygen delivery,May thicken blood,Red blood cells carry oxygen from your lungs t...
4,Platelet Count,150400,K/uL,May increase bleeding risk,May increase clotting risk,Platelets help your blood to clot and stop ble...


In [8]:
# ---------------------------------------------
# Rule-based Sinhala medical explanations
# ---------------------------------------------

def sinhala_medical_explanation(test, status, knowledge_row):
    """
    Generates SAFE Sinhala explanation without AI translation
    """

    base = knowledge_row["simple_explanation_en"]

    # Sinhala base explanations
    base_si = {
        "MCH": "MCH යනු රතු රුධිර කණිකා තුළ ඇති හීමෝග්ලොබින් ප්‍රමාණය මැනීමයි.",
        "MCV": "MCV යනු රතු රුධිර කණිකා වල සාමාන්‍ය ප්‍රමාණය පෙන්වයි.",
        "RDW": "RDW යනු රතු රුධිර කණිකා වල ප්‍රමාණ වෙනස් වීම පෙන්වයි.",
        "Platelet Count": "පලට්ලට් යනු රුධිරය කැටි වීමට උපකාරී වන කණිකා වේ.",
        "Cholesterol": "කොලෙස්ටරෝල් යනු ශරීරයට අවශ්‍ය තෙල් වර්ගයකි."
    }

    meaning_si = {
        "High": "මෙම අගය සාමාන්‍යයට වඩා ඉහළයි.",
        "Low": "මෙම අගය සාමාන්‍යයට වඩා අඩුයි.",
        "Normal": "මෙම අගය සාමාන්‍ය සීමාව තුළ ඇත."
    }

    explanation = f"""
{base_si.get(test, test + " රුධිර පරීක්ෂාවකි.")}

{meaning_si.get(status, "")}

මෙය අධ්‍යාපනික විස්තරයක් පමණක් වන අතර,
නිශ්චිත වෛද්‍ය උපදෙස් සඳහා වෛද්‍යවරයෙකු හමුවන්න.
"""

    return explanation.strip()


In [37]:
# -------------------------------
# Gemini setup (Chatbot module)
# -------------------------------
import google.generativeai as genai
from google.colab import userdata

# Configure API key
genai.configure(api_key=userdata.get("GEMINI_API_KEY"))

# Create Gemini model instance
model = genai.GenerativeModel("models/gemini-flash-lite-latest")

In [38]:
def fallback_chatbot_response(user_question, final_df):
    """
    Rule-based fallback when Gemini quota is exceeded.
    """
    for _, row in final_df.iterrows():
        if row["test_name"].lower() in user_question.lower():
            return (
                f"{row['test_name']} result is {row['value']} "
                f"and the status is {row['status']}. "
                "Please consult a doctor for more details."
            )
    return "I can only answer questions related to your blood report."


In [39]:
# ---------------------------------------------
# Build report context from final_df
# ---------------------------------------------

def build_report_context(final_df):
    """
    Converts final_df into readable medical context
    """
    context = ""

    for _, row in final_df.iterrows():
        context += (
            f"Test: {row['test_name']}, "
            f"Value: {row['value']}, "
            f"Status: {row['status']}. "
        )

    return context



In [40]:

def chatbot_prompt(user_question, report_context):
    """
    Safer + friendlier medical chatbot prompt
    """
    return f"""
You are a medical assistant chatbot.

Below is the patient's blood test report:
{report_context}

User Question:
{user_question}

Rules:
- Explain what the result means in general terms
- Do NOT give a diagnosis
- Do NOT prescribe medicine
- You MAY mention common general reasons (educational only)
- Use simple, patient-friendly language
- End by advising to consult a doctor
"""



In [41]:
# ---------------------------------------------
# XAI: Explain WHY Low / Normal / High
# ---------------------------------------------

def explain_classification(test_name, value, status, ref_low, ref_high):
    """
    Explainable AI logic (English)
    """
    if status == "Low":
        return {
            "reason": f"The value ({value}) is lower than the normal minimum ({ref_low})."
        }

    if status == "High":
        return {
            "reason": f"The value ({value}) is higher than the normal maximum ({ref_high})."
        }

    if status == "Normal":
        return {
            "reason": f"The value ({value}) falls within the normal range ({ref_low}–{ref_high})."
        }

    return {"reason": "Unable to explain this classification."}


In [46]:
# ---------------------------------------------
# XAI Sinhala (RULE-BASED, SAFE)
# ---------------------------------------------
def explain_classification_si(test_name, value, status):
    """
    Rich Sinhala Explainable AI explanation (educational, non-diagnostic)
    """

    base_definitions = {
        "MCH": (
            "MCH (Mean Corpuscular Hemoglobin) යනු "
            "රතු රුධිර කණිකා එක් එක් එකක ඇති "
            "හීමෝග්ලොබින් ප්‍රමාණය මැනීමයි.\n\n"
            "හීමෝග්ලොබින් යනු ශරීරයේ සියලුම කොටස් වෙත "
            "ඔක්සිජන් රැගෙන යන ප්‍රධාන ප්‍රෝටීනයකි."
        ),
        "MCV": (
            "MCV යනු රතු රුධිර කණිකා වල "
            "සාමාන්‍ය ප්‍රමාණය මැනීමයි."
        ),
        "RDW": (
            "RDW යනු රතු රුධිර කණිකා වල "
            "ප්‍රමාණ වෙනස් වීම පෙන්වන පරීක්ෂණයකි."
        ),
        "Cholesterol": (
            "Cholesterol යනු ශරීරයට අවශ්‍ය "
            "තෙල් වර්ගයක ප්‍රමාණය මැනීමයි."
        )
    }

    base_text = base_definitions.get(
        test_name,
        "මෙය රුධිර පරීක්ෂණයකි."
    )

    if status == "High":
        status_text = (
            "ඔබගේ වාර්තාව අනුව, මෙම අගය "
            "**සාමාන්‍යයට වඩා ඉහළ** ලෙස පෙන්වයි.\n\n"
            "සාමාන්‍යයෙන් මෙය පෙන්වන්නේ "
            "මෙම පරීක්ෂණයට අදාළ දේ "
            "සාමාන්‍යයට වඩා වැඩි ප්‍රමාණයකින් "
            "පවතින බවයි."
        )

    elif status == "Low":
        status_text = (
            "ඔබගේ වාර්තාව අනුව, මෙම අගය "
            "**සාමාන්‍යයට වඩා අඩුයි**.\n\n"
            "සාමාන්‍යයෙන් මෙය පෙන්වන්නේ "
            "මෙම පරීක්ෂණයට අදාළ දේ "
            "සාමාන්‍යයට වඩා අඩු ප්‍රමාණයකින් "
            "පවතින බවයි."
        )

    elif status == "Normal":
        status_text = (
            "මෙම අගය **සාමාන්‍ය සීමාව තුළ** පවතී.\n\n"
            "සාමාන්‍යයෙන් මෙය සෞඛ්‍යයට "
            "හිතකර තත්ත්වයක් ලෙස සැලකේ."
        )

    else:
        status_text = (
            "මෙම පරීක්ෂණය සඳහා "
            "පැහැදිලි තත්ත්වයක් "
            "තීරණය කිරීමට නොහැක."
        )

    return (
        f"{base_text}\n\n"
        f"{status_text}\n\n"
        "⚠️ මෙය අධ්‍යාපනික විස්තරයක් පමණක් වන අතර, "
        "නිශ්චිත වෛද්‍ය උපදෙස් සඳහා "
        "කරුණාකර වෛද්‍යවරයෙකු හමුවන්න."
    )



In [47]:

# ---------------------------------------------
# Bilingual Medical Chatbot with XAI
# ---------------------------------------------


def chatbot_response(user_question, final_df):
    """
    Bilingual Medical Chatbot
    Sinhala → Rule-based XAI
    English → Gemini + fallback
    """

    # Detect language
    user_lang = detect_language(user_question)

    # -------------------------------
    # SINHALA PATH (XAI – RULE BASED)
    # -------------------------------
    if user_lang == "si":

        for _, row in final_df.iterrows():

            if row["test_name"].lower() in user_question.lower():

                xai_text = explain_classification_si(
                test_name=row["test_name"],
                value=row["value"],
                status=row["status"]
            )


                return (
                    f"{row['test_name']} යනු රුධිර පරීක්ෂණයකි.\n\n"
                    f"අගය: {row['value']}\n"
                    f"තත්ත්වය: {row['status']}\n\n"
                    f"{xai_text}\n\n"
                    "මෙය අධ්‍යාපනික විස්තරයක් පමණක් වන අතර, "
                    "නිශ්චිත වෛද්‍ය උපදෙස් සඳහා වෛද්‍යවරයෙකු හමුවන්න."
                )

        return "ඔබගේ වාර්තාවට අදාල ප්‍රශ්නයක් අසන්න."

    # -------------------------------
    # ENGLISH PATH (GEMINI)
    # -------------------------------
    try:
        report_context = build_report_context(final_df)
        prompt = chatbot_prompt(user_question, report_context)
        response = model.generate_content(prompt)
        return response.text

    except:
        return fallback_chatbot_response(user_question, final_df)





In [45]:
#Test english
print(chatbot_response("Why is my MCH high?", final_df))


I see you have a question about your recent blood test results, specifically why your **MCH (Mean Corpuscular Hemoglobin)** is high.

Here is a simple explanation:

**What MCH Measures:**
The MCH is a measurement that tells us the **average amount (weight) of hemoglobin** inside your individual red blood cells. Hemoglobin is the protein in your red blood cells that carries oxygen throughout your body.

**What a High MCH Means:**
When your MCH value is high, it generally suggests that your red blood cells are carrying **more hemoglobin than usual** on average. This often means that your red blood cells are larger than normal (which might also be reflected in your MCV, though your MCV in this report appears normal).

**Common General Reasons (Educational Only):**
While I cannot provide a diagnosis, in general, high MCH values can sometimes be associated with conditions where the body produces larger-than-normal red blood cells (macrocytosis). Common general reasons for this can sometimes

In [48]:
#Test sinhala
print(chatbot_response("මගේ MCH ඉහළ ඇයි?", final_df))


MCH යනු රුධිර පරීක්ෂණයකි.

අගය: 287.0
තත්ත්වය: High

MCH (Mean Corpuscular Hemoglobin) යනු රතු රුධිර කණිකා එක් එක් එකක ඇති හීමෝග්ලොබින් ප්‍රමාණය මැනීමයි.

හීමෝග්ලොබින් යනු ශරීරයේ සියලුම කොටස් වෙත ඔක්සිජන් රැගෙන යන ප්‍රධාන ප්‍රෝටීනයකි.

ඔබගේ වාර්තාව අනුව, මෙම අගය **සාමාන්‍යයට වඩා ඉහළ** ලෙස පෙන්වයි.

සාමාන්‍යයෙන් මෙය පෙන්වන්නේ මෙම පරීක්ෂණයට අදාළ දේ සාමාන්‍යයට වඩා වැඩි ප්‍රමාණයකින් පවතින බවයි.

⚠️ මෙය අධ්‍යාපනික විස්තරයක් පමණක් වන අතර, නිශ්චිත වෛද්‍ය උපදෙස් සඳහා කරුණාකර වෛද්‍යවරයෙකු හමුවන්න.

මෙය අධ්‍යාපනික විස්තරයක් පමණක් වන අතර, නිශ්චිත වෛද්‍ය උපදෙස් සඳහා වෛද්‍යවරයෙකු හමුවන්න.


In [50]:
print(chatbot_response("Is my cholesterol normal?", final_df))


Based on your blood test report, your **Cholesterol value is 194.0**, and the **Status is Normal**.

In general terms, a normal cholesterol level is important for your body because cholesterol is a waxy substance needed to build healthy cells. When levels are within the normal range, it usually means your risk for heart-related issues associated with high cholesterol is not increased by this measurement.

If levels are too high, it can sometimes be related to diet, lack of exercise, or genetics, but please remember that this result is just one piece of information.

**It is important to discuss all of your test results, including this cholesterol reading, with your doctor** for a complete understanding of your health picture.


In [49]:
print(chatbot_response("මගේ කොලෙස්ටරෝල් සාමාන්‍යද?", final_df))


ඔබගේ වාර්තාවට අදාල ප්‍රශ්නයක් අසන්න.


In [51]:
print(chatbot_response("RDW low කියන්නේ මොකක්ද?", final_df))


RDW යනු රුධිර පරීක්ෂණයකි.

අගය: 8.0
තත්ත්වය: Low

RDW යනු රතු රුධිර කණිකා වල ප්‍රමාණ වෙනස් වීම පෙන්වන පරීක්ෂණයකි.

ඔබගේ වාර්තාව අනුව, මෙම අගය **සාමාන්‍යයට වඩා අඩුයි**.

සාමාන්‍යයෙන් මෙය පෙන්වන්නේ මෙම පරීක්ෂණයට අදාළ දේ සාමාන්‍යයට වඩා අඩු ප්‍රමාණයකින් පවතින බවයි.

⚠️ මෙය අධ්‍යාපනික විස්තරයක් පමණක් වන අතර, නිශ්චිත වෛද්‍ය උපදෙස් සඳහා කරුණාකර වෛද්‍යවරයෙකු හමුවන්න.

මෙය අධ්‍යාපනික විස්තරයක් පමණක් වන අතර, නිශ්චිත වෛද්‍ය උපදෙස් සඳහා වෛද්‍යවරයෙකු හමුවන්න.
